#  Craw OrangeTour

In [2]:
import requests_html
import json, time, re

In [3]:
session = requests_html.HTMLSession()

In [4]:
# 建立dict用來轉換對方的欄位轉成自己的欄位
info_dict={'GrupCd': 'id',
           'GrupSnm':'name',
           'GrupLn':'days',
           'LeavDt':'date',
           'SaleAm':'price',
           'SaleYqt':'available',
           'EstmTotqt':'total'
          }

In [5]:
# 設定目標頁面
url='http://www.orangetour.com.tw/EW/Services/SearchListData.asp'

In [6]:
# 建立空的list準備放各種行程
products=[]

# 用迴圈跑需要的頁數
for page in range(1, 3):
    # 抓存放行程資料的json檔
    f = session.post(url, data={'pageALL':page})
    # 讀json檔, 行程內容都存放在All裡面, 把它們抓出來
    all_products = json.loads(f.text)['All']
    
    # 跑回圈取出每一個行程, 並把需要的欄位轉成我們自己的欄位取出來
    for all_product in all_products:
        product_dic={}
        for info in info_dict.keys():
            product_dic[info_dict[info]] = all_product[info]
        products.append(product_dic)
    
    # 作為一個友善的爬蟲程式, 我們會sleep just like a human
    time.sleep(5)

In [21]:
# id是唯一且不重複的, 對它取set可以確保我們沒有抓到重複的資料
ids = [product['id'] for product in products]
assert len(set(ids))==len(products)

In [9]:
# 印出我們抓下的資料內容
for product in products:
    for info in product.keys():
        print(str(info), str(product[info]), sep=' : ')
    print('==========================')

id : ICN180831KEE
name : 超值熱賣哈韓5日遊
days : 5
date : 2018/08/31
price : 14800
available : 19
total : 19
id : AS0480831ZZA
name : “午夜陽光” 阿拉斯加夏季4天3夜經典系列
days : 4
date : 2018/08/31
price : 20000
available : 4
total : 4
id : EE1080831BRA
name : 奧捷雙國、米其林、湖區、古堡、遊船、雙登塔10天(長榮直飛)
days : 10
date : 2018/08/31
price : 62900
available : 25
total : 26
id : SS1080831TKA
name : 西班牙 七大城 聖家堂 佛朗明哥晚餐秀10日
days : 10
date : 2018/08/31
price : 69900
available : 25
total : 26
id : EW0980901CAA
name : 【雙點進出+三大名峰】峰花雪嶽 瑞士9日
days : 9
date : 2018/09/01
price : 69900
available : 0
total : 26
id : AK0880901KQA
name : 肯亞 我要玩狠大8天
days : 8
date : 2018/09/01
price : 125900
available : 12
total : 13
id : ME1180903TKA
name : 5星【直飛】加贈洞穴飯店 蕃紅花城11日
days : 11
date : 2018/09/03
price : 47900
available : 0
total : 22
id : EW1080903BRA
name : Hello! 長榮荷德比法 3遊船 OUTLET 10天
days : 10
date : 2018/09/03
price : 69900
available : 0
total : 26
id : ES1580904TKA
name : 南歐伊比利半島西班牙、葡萄牙、直布羅陀、畢爾包浪漫美食15天
days : 15
date : 2018/09/04
price : 1249

In [10]:
# 抓下網頁第一頁的資料
f = session.get('http://www.orangetour.com.tw/EW/GO/GroupList.asp')

In [11]:
# 抓網頁上的行程
thumbnails = f.html.find('.thumbnail', first=False)

In [12]:
# 驗證資料是否與網頁相同
for product in products:
    print(product['id'], product['name'])
    
    for thumbnail in thumbnails:
        if not re.search(product['id'], thumbnail.find('.product_num', first=True).text):
            continue
        
        if not re.search(product['date'], thumbnail.find('.product_date', first=True).text):
            raise Exception(product['date']+thumbnail.find('.product_date', first=True).text)
        
        if not re.search(str(product['days']), thumbnail.find('.product_days', first=True).text):
            raise Exception(str(product['days'])+thumbnail.find('.product_days', first=True).text)
        
        if not re.search(format(product['price'],','), thumbnail.find('.product_price', first=True).text):
            raise Exception(format(product['price'],',')+thumbnail.find('.product_price', first=True).text)
        
        if not re.search(str(product['available']), thumbnail.find('.product_available', first=True).text):
            raise Exception(str(product['available'])+thumbnail.find('.product_available', first=True).text)
        
        if not re.search(str(product['total']), thumbnail.find('.product_total', first=True).text):
            raise Exception(str(product['total'])+thumbnail.find('.product_total', first=True).text)

ICN180831KEE 超值熱賣哈韓5日遊
AS0480831ZZA “午夜陽光” 阿拉斯加夏季4天3夜經典系列
EE1080831BRA 奧捷雙國、米其林、湖區、古堡、遊船、雙登塔10天(長榮直飛)
SS1080831TKA 西班牙 七大城 聖家堂 佛朗明哥晚餐秀10日
EW0980901CAA 【雙點進出+三大名峰】峰花雪嶽 瑞士9日
AK0880901KQA 肯亞 我要玩狠大8天
ME1180903TKA 5星【直飛】加贈洞穴飯店 蕃紅花城11日
EW1080903BRA Hello! 長榮荷德比法 3遊船 OUTLET 10天
ES1580904TKA 南歐伊比利半島西班牙、葡萄牙、直布羅陀、畢爾包浪漫美食15天
EN1280904SKB 榮耀斯堪地那維亞之旅-北歐五國、冰島、峽灣觀景列車經典12日
ICN180905KEB 親子水陸雙樂園5日
ICN180905KEA 哈韓94狂～超人氣三大主題5日
EC1280905BRA 精緻千禧號遊輪-阿拉斯加哈伯冰河南下航線12天
TYO180906ITA 東京蝦拼半自助漫遊五日
EE1280906OSA 5星巴爾幹半島 克斯蒙3國 雙跳島、雙國家公園、三遊船12天
AK1080906KQA 肯亞 野性呼喚10天 非洲王子系列
EE1080907BRA 奧捷雙國、米其林、湖區、古堡、遊船、雙登塔10天(長榮直飛)
SS1080907TKA 西班牙 七大城 聖家堂 佛朗明哥晚餐秀10日
ME1080908TKA 直飛土耳其世界遺產、愛琴海蕃紅花10日(5晚5星+1晚洞穴)
AK0880908KQA 肯亞 我要玩狠大8天
EW1080909QRA 品味法拉利 南北義 龐貝 藍洞 OUTLET 10天
ES1080909CAA 浪漫愛琴海雙島：聖托里尼、米克諾斯、天空之城、諸神的故鄉、希臘10天之旅
CSX180910CZC 張家界天門山袁家界鳳凰古城玻璃橋精選8日
LHW180910HUA 迴響夢駝鈴系列〜河西走廊五彩山、莫高窟、鳴沙山+月牙泉8日
EW1080901BRA Hello! 長榮荷德比法 3遊船 OUTLET 10天
DYG180911CZB 美麗新視界~張家界鳳凰古城6天
ICN180912KEB 親子水陸雙樂園5日
ICN180912KEA 哈韓94狂～超人氣三大主題5日
TXN180912BRC 